In [13]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

df = pd.read_csv('data.csv')

# возьмем перые 100 строк, 4-й столбец 
y = df.iloc[0:100, 4].values
# так как ответы у нас строки - нужно перейти к численным значениям
y = np.where(y == "Iris-setosa", 1, 0).reshape(-1,1) # reshape нужен для матричных операций

# возьмем два признака, чтобы было удобне визуализировать задачу
X = df.iloc[0:100, [0, 2]].values

# добавим фиктивный признак для удобства матричных вычслений
X = np.concatenate([np.ones((len(X),1)), X], axis = 1)

# зададим функцию активации - сигмоида
def sigmoid(y):
    return 1 / (1 + np.exp(-y))

# нам понадобится производная от сигмоиды при вычислении градиента
def derivative_sigmoid(y):
    return sigmoid(y) * (1 - sigmoid(y))


# инициализируем нейронную сеть 
inputSize = X.shape[1] # количество входных сигналов равно количеству признаков задачи 
hiddenSizes = 5 # задаем число нейронов скрытого слоя 
outputSize = 1 if len(y.shape) else y.shape[1] # количество выходных сигналов равно количеству классов задачи

# веса инициализируем случайными числами, но теперь будем хранить их списком
weights = [
    np.random.uniform(-2, 2, size=(inputSize,hiddenSizes)),  # веса скрытого слоя
    np.random.uniform(-2, 2, size=(hiddenSizes,outputSize))  # веса выходного слоя
]

# прямой проход 
def feed_forward(x):
    input_ = x # входные сигналы
    hidden_ = sigmoid(np.dot(input_, weights[0])) # выход скрытого слоя = сигмоида(входные сигналы*веса скрытого слоя)
    output_ = sigmoid(np.dot(hidden_, weights[1]))# выход сети (последнего слоя) = сигмоида(выход скрытого слоя*веса выходного слоя)

    # возвращаем все выходы, они нам понадобятся при обратном проходе
    return [input_, hidden_, output_]

# backprop собственной персоной
# на вход принимает скорость обучения, реальные ответы, предсказанные сетью ответы и выходы всех слоев после прямого прохода
def backward(learning_rate, target, net_output, layers):

    # считаем производную ошибки сети
    err = (target - net_output)

    # прогоняем производную ошибки обратно ко входу, считая градиенты и корректируя веса
    # для этого используем chain rule
    # цикл перебирает слои от последнего к первому
    for i in range(len(layers)-1, 0, -1):
        # градиент слоя = ошибка слоя * производную функции активации * на входные сигналы слоя
        
        # ошибка слоя * производную функции активации
        err_delta = err * derivative_sigmoid(layers[i])       
        
        # пробрасываем ошибку на предыдущий слой
        err = np.dot(err_delta, weights[i - 1].T)
        
        # ошибка слоя * производную функции активации * на входные сигналы слоя
        dw = np.dot(layers[i - 1].T, err_delta)
        
        # обновляем веса слоя
        weights[i - 1] += learning_rate * dw
        
        

# функция обучения чередует прямой и обратный проход
def train(x_values, target, learning_rate):
    output = feed_forward(x_values)
    backward(learning_rate, target, output[2], output)
    return None

# функция предсказания возвращает только выход последнего слоя
def predict(x_values):
    return feed_forward(x_values)[-1]


# задаем параметры обучения
iterations = 50
learning_rate = 0.01

# обучаем сеть (фактически сеть это вектор весов weights)
for i in range(iterations):
    train(X, y, learning_rate)

    if i % 10 == 0:
        print("На итерации: " + str(i) + ' || ' + "Средняя ошибка: " + str(np.mean(np.square(y - predict(X)))))

# считаем ошибку на обучающей выборке
pr = predict(X)
print(sum(abs(y-(pr>0.5))))


# считаем ошибку на всей выборке
y = df.iloc[:, 4].values
y = np.where(y == "Iris-setosa", 1, 0).reshape(-1,1) 
X = df.iloc[:, [0, 2]].values
X = np.concatenate([np.ones((len(X),1)), X], axis = 1)

pr = predict(X)
print(sum(abs(y-(pr>0.5))))



На итерации: 0 || Средняя ошибка: 0.25725900348484354
На итерации: 10 || Средняя ошибка: 0.24807772973577258
На итерации: 20 || Средняя ошибка: 0.2487266322755746
На итерации: 30 || Средняя ошибка: 0.2495037089679268
На итерации: 40 || Средняя ошибка: 0.2498283814911141
[50]
[50]


In [15]:
df.head(10)

,0,1,2,3,Class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
5,5.4,3.9,1.7,0.4,Iris-setosa
6,4.6,3.4,1.4,0.3,Iris-setosa
7,5.0,3.4,1.5,0.2,Iris-setosa
8,4.4,2.9,1.4,0.2,Iris-setosa
9,4.9,3.1,1.5,0.1,Iris-setosa


In [16]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

df = pd.read_csv('data.csv')

# возьмем первые 100 строк, 4-й столбец 
y = df.iloc[0:100, 4].values
# так как ответы у нас строки - нужно перейти к численным значениям
y = np.where(y == "Iris-setosa", 1, 0).reshape(-1, 1)  # reshape нужен для матричных операций

# возьмем два признака, чтобы было удобно визуализировать задачу
X = df.iloc[0:100, [0, 2]].values

# добавим фиктивный признак для удобства матричных вычислений
X = np.concatenate([np.ones((len(X), 1)), X], axis=1)

# зададим функцию активации - сигмоида
def sigmoid(y):
    return 1 / (1 + np.exp(-y))

# нам понадобится производная от сигмоиды при вычислении градиента
def derivative_sigmoid(y):
    return sigmoid(y) * (1 - sigmoid(y))

# инициализируем нейронную сеть 
inputSize = X.shape[1]  # количество входных сигналов равно количеству признаков задачи 
hiddenSizes = 5  # задаем число нейронов скрытого слоя 
outputSize = 1 if len(y.shape) else y.shape[1]  # количество выходных сигналов равно количеству классов задачи

# веса инициализируем случайными числами, но теперь будем хранить их списком
weights = [
    np.random.uniform(-2, 2, size=(inputSize, hiddenSizes)),  # веса скрытого слоя
    np.random.uniform(-2, 2, size=(hiddenSizes, outputSize))  # веса выходного слоя
]

# стохастическое обучение
def stochastic_train(X, y, learning_rate, epochs):
    for epoch in range(epochs):
        random_indices = np.arange(len(X))
        np.random.shuffle(random_indices)
        X = X[random_indices]
        y = y[random_indices]
        for i in range(len(X)):
            x_values = X[i].reshape(1, -1)
            target = y[i].reshape(1, -1)
            
            # прямой проход
            input_, hidden_, output_ = feed_forward(x_values)

            # обратный проход
            backward(learning_rate, target, output_, [input_, hidden_, output_])

        if epoch % 10 == 0:
            print("На эпохе:", epoch, '||', "Средняя ошибка:", np.mean(np.square(y - predict(X))))

# функция обучения чередует прямой и обратный проход
#def train(x_values, target, learning_rate):
#    output = feed_forward(x_values)
#    backward(learning_rate, target, output[2], output)
#    return None

# функция предсказания возвращает только выход последнего слоя
def predict(x_values):
    return feed_forward(x_values)[-1]


# задаем параметры обучения
epochs = 50
learning_rate = 0.01

# обучаем сеть (фактически сеть это вектор весов weights)
stochastic_train(X, y, learning_rate, epochs)

# считаем ошибку на обучающей выборке
pr = predict(X)
print(sum(abs(y - (pr > 0.5))))

# считаем ошибку на всей выборке
y = df.iloc[:, 4].values
y = np.where(y == "Iris-setosa", 1, 0).reshape(-1, 1) 
X = df.iloc[:, [0, 2]].values
X = np.concatenate([np.ones((len(X), 1)), X], axis=1)

pr = predict(X)
print(sum(abs(y - (pr > 0.5))))

На эпохе: 0 || Средняя ошибка: 0.35347388347757547
На эпохе: 10 || Средняя ошибка: 0.2682648902245271
На эпохе: 20 || Средняя ошибка: 0.25459637023018816
На эпохе: 30 || Средняя ошибка: 0.24771728008159655
На эпохе: 40 || Средняя ошибка: 0.23135324466295604
[50]
[50]


In [20]:
import numpy as np
import pandas as pd

# Загрузим данные
df = pd.read_csv('data.csv')

# Преобразуем метки классов в численные значения
class_mapping = {'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}
df['Class'] = df['Class'].map(class_mapping)

# Разделим данные на признаки и метки
X = df.iloc[:, :4].values
y = df['Class'].values.reshape(-1, 1)
#print(y)
# Добавим фиктивный признак для удобства матричных вычислений
X = np.concatenate([np.ones((len(X), 1)), X], axis=1)

# Зададим функцию активации - сигмоида
def sigmoid(y):
    return 1 / (1 + np.exp(-y))

# Производная от сигмоиды для вычисления градиента
def derivative_sigmoid(y):
    return sigmoid(y) * (1 - sigmoid(y))

# Инициализируем нейронную сеть
input_size = X.shape[1]  # Количество входных сигналов равно количеству признаков задачи 
hidden_size = 5  # Задаем количество нейронов в скрытом слое 
output_size = 3  # Количество выходных сигналов равно количеству классов задачи

# Веса инициализируем случайными числами
weights = [
    np.random.uniform(-2, 2, size=(input_size, hidden_size)),  # Веса скрытого слоя
    np.random.uniform(-2, 2, size=(hidden_size, output_size))   # Веса выходного слоя
]

# Прямой проход
def feed_forward(x):
    input_ = x
    hidden_ = sigmoid(np.dot(input_, weights[0]))
    output_ = sigmoid(np.dot(hidden_, weights[1]))
    return [input_, hidden_, output_]

# Обратный проход
def backward(learning_rate, target, net_output, layers):
    err = (target - net_output)
    
    for i in range(len(layers) - 1, 0, -1):
        err_delta = err * derivative_sigmoid(layers[i])
        err = np.dot(err_delta, weights[i - 1].T)
        dw = np.dot(layers[i - 1].T, err_delta)
        weights[i - 1] += learning_rate * dw

# Функция обучения
def train(x_values, target, learning_rate):
    output = feed_forward(x_values)
    backward(learning_rate, target, output[2], output)
    return None

# Функция предсказания
def predict(x_values):
    return feed_forward(x_values)[-1]

# Параметры обучения
epochs = 1000
learning_rate = 0.01

# Обучаем сеть
for epoch in range(epochs):
    random_indices = np.arange(len(X))
    np.random.shuffle(random_indices)
    X = X[random_indices]
    y = y[random_indices]
    for i in range(len(X)):
        x_values = X[i].reshape(1, -1)
        target = np.zeros((1, output_size))
        target[0, y[i]] = 1  # One-hot encoding меток классов
        train(x_values, target, learning_rate)

# Считаем точность на обучающей выборке
correct_predictions = 0
for i in range(len(X)):
    x_values = X[i].reshape(1, -1)
    predicted_class = np.argmax(predict(x_values))
    if predicted_class == y[i]:
        correct_predictions += 1

accuracy = correct_predictions / len(X)
print(f"Точность на обучающей выборке: {accuracy * 100:.2f}%")



Точность на обучающей выборке: 72.00%


In [21]:
X = df.iloc[100:101, :4].values
# Добавим фиктивный признак для удобства матричных вычислений
X = np.concatenate([np.ones((len(X), 1)), X], axis=1)
y_pred = predict(X)
y_pred

array([[0.0020472 , 0.4228473 , 0.57105798]])

In [10]:
target

array([[0., 0., 1.]])